In [3]:
import os
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import PIL
import random

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Dense, MaxPool2D, Flatten, LeakyReLU, BatchNormalization, Dropout, Input

%matplotlib inline

base_path = r'C:\\Users\Computing\Downloads\data-task1\data-task1'
test_path = os.path.join(base_path,'test')
train_path = os.path.join(base_path,'train')
val_path = os.path.join(base_path, 'val')

try:
    os.mkdir("/kaggle/working/models")
except:
    pass

In [4]:
IMAGE_SIZE=(224,224)
BATCH_SIZE=64

train_datagen  = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary')

test_datagen  = ImageDataGenerator(
    rescale=1./255,
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary')

Found 3000 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [8]:
model1 = tf.keras.models.Sequential([
    Input(shape=(224, 224, 3,)),
    
    Conv2D(64, 5, 2),
    LeakyReLU(),
    MaxPool2D(2),
    #BatchNormalization(),
    #Dropout(),

    Conv2D(128, 3, 2),
    LeakyReLU(),
    MaxPool2D(2),
    #BatchNormalization(),
    #Dropout(),

    Conv2D(256, 3, 1, padding='same'),
    LeakyReLU(),
    MaxPool2D(2),
    #BatchNormalization(),
    #Dropout(),

    Conv2D(512, 3, 1, padding='same'),
    LeakyReLU(),
    MaxPool2D(2),
    #BatchNormalization(),
    #Dropout(),
    
    Flatten(),
    Dense(128),
    LeakyReLU(),
    Dense(64),
    LeakyReLU(),
    Dense(1, activation='sigmoid')    
])

In [9]:
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')


In [10]:
model2 = tf.keras.models.Sequential([
    Input(shape=(224, 224, 3,)),
    
    Conv2D(64, 5, 2),
    LeakyReLU(),
    MaxPool2D(2),
    BatchNormalization(),
    Dropout(0.2),

    Conv2D(128, 3, 2),
    LeakyReLU(),
    MaxPool2D(2),
    BatchNormalization(),
    Dropout(0.2),

    Conv2D(256, 3, 1, padding='same'),
    LeakyReLU(),
    MaxPool2D(2),
    BatchNormalization(),

    Conv2D(512, 3, 1, padding='same'),
    LeakyReLU(),
    MaxPool2D(2),
    BatchNormalization(),
    
    Flatten(),
    Dense(128),
    LeakyReLU(),
    Dense(64),
    LeakyReLU(),
    Dense(1, activation='sigmoid')    
])

In [11]:
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')

In [12]:
def prepare_model(model, input_shape=(224, 224, 3), optimizer='adam'):
    pre_model = model(input_shape=input_shape,
                 include_top=False,
                 weights='imagenet')
    
    for layer in pre_model.layers:
        layer.trainable = False
        
    last_out = pre_model.layers[-1].output
    x = Flatten()(last_out) 
    x = Dense(512, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(pre_model.input, x)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics='accuracy')
    
    return model   

In [13]:
def train_model(model, name, train, test, epochs=50):
    callbacks =[] 
    callbacks.append(tf.keras.callbacks.EarlyStopping(patience=5))
    callbacks.append(tf.keras.callbacks.ModelCheckpoint(os.path.join('/kaggle/working/models', name), save_best_only=True))
    
    history = model.fit(train, validation_data=test, epochs=epochs, callbacks=callbacks)
    
    return model, name, history

In [14]:
models_to_prepare = [
    (tf.keras.applications.inception_v3.InceptionV3, 'inception'),
    (tf.keras.applications.ResNet50, 'resnet'),
    (tf.keras.applications.vgg16.VGG16, 'vgg'),
    (tf.keras.applications.xception.Xception, 'xception'),
]

models = [(prepare_model(model[0]), model[1]) for model in models_to_prepare]

models += [
    (model1, 'leaky_bachnorm'),
    (model2, 'leaky'),
]

83697664/83683744 [==============================] - 1s 0us/step


In [ ]:
trained_models = []
histories = []
for model in models:
    model, name, history = train_model(model[0], model[1], train_generator, test_generator, epochs=50) 
    trained_models.append(model)
    histories.append((name, history))

Epoch 1/50
47/47 [==============================] - 96s 2s/step - loss: 0.9959 - accuracy: 0.7713 - val_loss: 0.5414 - val_accuracy: 0.8000
Epoch 2/50
47/47 [==============================] - 84s 2s/step - loss: 0.5416 - accuracy: 0.7977 - val_loss: 0.5472 - val_accuracy: 0.7760
Epoch 3/50
47/47 [==============================] - 82s 2s/step - loss: 0.7265 - accuracy: 0.7823 - val_loss: 0.8330 - val_accuracy: 0.8080
Epoch 4/50
47/47 [==============================] - 83s 2s/step - loss: 0.5484 - accuracy: 0.8080 - val_loss: 0.9514 - val_accuracy: 0.6840
Epoch 5/50
47/47 [==============================] - 83s 2s/step - loss: 0.4811 - accuracy: 0.8283 - val_loss: 0.5462 - val_accuracy: 0.7880
Epoch 6/50
47/47 [==============================] - 84s 2s/step - loss: 0.4505 - accuracy: 0.8280 - val_loss: 0.5707 - val_accuracy: 0.8100
Epoch 1/50
47/47 [==============================] - 128s 3s/step - loss: 4.2441 - accuracy: 0.6340 - val_loss: 0.5121 - val_accuracy: 0.7900
Epoch 2/50
47/47 [=